# Bibliotheksstatistik

Die [Deutsche Bibliotheksstatistik](https://www.bibliotheksstatistik.de/) kann frei abgerufen werden.

Für die Auswertung habe ich den Zeitraum von **2010 bis 2020** gewählt.

Folgende Variablen habe ich ausgewählt:
- Ausgaben Erwerbungen
- Bestand (virtuell)
- Bestand (insgesamt)
- Entleihende (ab 60 J.)
- Entleihende (bis 12 J.)
- Entleihende (insgesamt)
- Entleihen (virtueller Bestand)
- Entleihen (gesamt)
- Laufende Ausgaben

Das Ergebnis habe ich als **csv-Datei** heruntergeladen.

## Daten aufbereiten
### Daten importieren

In [74]:
# import modules
import pandas as pd

df = pd.read_csv('./data/1_dbs_data.csv', delimiter=';', header=[2,3], encoding='ISO-8859-1', low_memory=False)

print(df.head())

                  NR               Name  \
  Unnamed: 0_level_1 Unnamed: 1_level_1   
0                  1            Aach GB   
1                  2          Aachen GB   
2                  3          Aachen GB   
3                  4          Aachen GB   
4                  5          Aachen GB   

                                          Bibliothek             DBS-ID  \
                                  Unnamed: 2_level_1 Unnamed: 3_level_1   
0            Kath. Öffentliche Bücherei St. Nikolaus              QN043   
1            Kath. Öffentliche Bibliothek St. Konrad              PY065   
2        Kath. Öffentliche Bibliothek St. Laurentius              QU719   
3           Kath. Öffentliche Bibliothek St. Severin              NH551   
4  Bücherinsel Vaalser Straße der Pfarrgemeinde S...              QN262   

  Entleiher                           ... Ausg. Erwerbung                \
       2010 2011 2012 2013 2014 2015  ...            2012   2013   2014   
0       NaN  NaN  NaN 

### Nur die relevanten Daten im Dataframe speichern
Im Dataframe sind nun alle Daten aus den verschiedenen öffentlichen Bibliotheken in Deutschland gesammelt. Ich brauche jedoch nur die Summe, also die Daten für ganz Deutschland. Diese befinden sich in einer Zeile namens *Summe*. Deshalb übernehme ich nur diese.

In [75]:
sum_row = df.loc[(df[('NR','Unnamed: 0_level_1')] == 'Summe')]

### Tausender- und Dezimaltrennzeichen ersetzen
Die Zahlen haben Punkte als Tausendertrennzeichen. Diese müssen entfernt werden. Dafür wandle ich sie in Strings um, damit ich ```str.replace()``` anwenden kann. Als Dezimaltrennzeichen ersetze ich das Komma durch einen Punkt.

In [76]:
library_df = sum_row.astype('string')

for label, content in library_df.iteritems():
    library_df[label] = library_df[label].str.replace('.', '', regex=True)
    library_df[label] = library_df[label].str.replace(',', '.', regex=True)

library_df

NR               Name         Bibliothek  \
     Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
8547              Summe               <NA>               <NA>   

                 DBS-ID Entleiher                                      \
     Unnamed: 3_level_1      2010     2011     2012     2013     2014   
8547               <NA>   7844530  7725936  7604293  7479921  7355708   

               ... Ausg. Erwerbung                                   \
         2015  ...            2012       2013       2014       2015   
8547  7249936  ...       100690622  104213606  104353361  104934760   

                                                                   \
           2016       2017       2018          2019          2020   
8547  108944525  109927695  111316147  114191003.07  111839663.17   

     Unnamed: 103_level_0  
     Unnamed: 103_level_1  
8547                 <NA>  

[1 rows x 104 columns]

### Spalten ändern
Die Spaltennamen sind immer in eine Variable wie beispielsweise *Ausgaben Erwerbung* und dann in die Jahre eingeteilt. Ich möchte jedoch, dass jedes Jahr eine Zeile ist und jede Variable eine Spalte. So ist es besser lesbar. Dafür drehe

In [77]:
library_df = pd.melt(library_df)
library_df = library_df.rename(columns={"variable_0": "Auspraegung", 'variable_1': 'Jahr', 'value': 'Wert'})
library_df

,Auspraegung,Jahr,Wert
0,NR,Unnamed: 0_level_1,Summe
1,Name,Unnamed: 1_level_1,<NA>
2,Bibliothek,Unnamed: 2_level_1,<NA>
3,DBS-ID,Unnamed: 3_level_1,<NA>
4,Entleiher,2010,7844530
...,...,...,...
99,Ausg. Erwerbung,2017,109927695
100,Ausg. Erwerbung,2018,111316147
101,Ausg. Erwerbung,2019,114191003.07
102,Ausg. Erwerbung,2020,111839663.17


Einige Zeilen beinhalten irrelevante Daten.

In [78]:
library_df = library_df.drop([0, 1, 2, 3, 103])

Nun drehe ich das Dataframe so, dass das Jahr den Index darstellt und die Variablen die Spalten.

In [79]:
library_df = library_df.pivot(index='Jahr', columns='Auspraegung', values='Wert')

print(library_df)

Auspraegung Ausg. Erwerbung Best. virt.Best. Bestand insges Entl. ab 60 J.  \
Jahr                                                                         
2010               96842615           457360      123027730         737336   
2011               99526558           591069      122796523         760140   
2012              100690622           829951      122025276         805601   
2013              104213606          1265499      121235283         819270   
2014              104353361          1622216      119858476         846045   
2015              104934760          1134853      117511823         868571   
2016              108944525          1491330      116080149         919600   
2017              109927695          1305260      113808636         932875   
2018              111316147          1780716      112176481         970320   
2019           114191003.07          2977275      110813254         999957   
2020           111839663.17          4440515      108085305     

### Werte in Floats umwandeln
Damit mit den Werten später gerechnet werden kann, müssen sie noch in Floats umgewandelt werden.

In [80]:
for label, content in library_df.iteritems():
    library_df[label] = library_df[label].astype(float)

### Index als Spalte

In [81]:
library_df.reset_index(inplace=True)

## Bereinigte Daten

In [82]:
library_df

Auspraegung,Jahr,Ausg. Erwerbung,Best. virt.Best.,Bestand insges,Entl. ab 60 J.,Entl. bis 12 J.,Entl. virt.Best.,Entleih. insges.,Entleiher,Lfd. Ausgaben
0,2010,9.684262e+07,457360.0,123027730.0,737336.0,2076475.0,961683.0,375793896.0,7844530.0,8.222172e+08
1,2011,9.952656e+07,591069.0,122796523.0,760140.0,2081832.0,1443091.0,378719659.0,7725936.0,8.459510e+08
2,2012,1.006906e+08,829951.0,122025276.0,805601.0,2034793.0,3418398.0,375565974.0,7604293.0,8.582293e+08
3,2013,1.042136e+08,1265499.0,121235283.0,819270.0,2015921.0,7060821.0,373419034.0,7479921.0,8.748400e+08
4,2014,1.043534e+08,1622216.0,119858476.0,846045.0,2003788.0,11436854.0,362542582.0,7355708.0,9.021050e+08
5,2015,1.049348e+08,1134853.0,117511823.0,868571.0,1973670.0,16314414.0,361763984.0,7249936.0,9.218251e+08
6,2016,1.089445e+08,1491330.0,116080149.0,919600.0,2034986.0,19813365.0,355303531.0,7431335.0,9.484614e+08
7,2017,1.099277e+08,1305260.0,113808636.0,932875.0,2028492.0,24350279.0,344544584.0,7306049.0,9.462734e+08
8,2018,1.113161e+08,1780716.0,112176481.0,970320.0,2063797.0,28979729.0,338358540.0,7264051.0,9.712075e+08
9,2019,1.141910e+08,2977275.0,110813254.0,999957.0,2108134.0,33410061.0,338357207.0,7309666.0,1.006269e+09


In [83]:
library_df.dtypes

Auspraegung
Jahr                 object
Ausg. Erwerbung     float64
Best. virt.Best.    float64
Bestand insges      float64
Entl. ab 60 J.      float64
Entl. bis 12 J.     float64
Entl. virt.Best.    float64
Entleih. insges.    float64
Entleiher           float64
Lfd. Ausgaben       float64
dtype: object